In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from functools import partial
from utils import chart_price, plot, v_backtester, c_backtester, perf, perf_var, breakout_strategy, bootstrap, m_proc
import sys
sys.path.append('/home/tomek/ib_tools')
from indicators import get_ATR, get_signals
import matplotlib.pyplot as plt
from typing import NamedTuple
from collections import namedtuple
%matplotlib inline

In [2]:
from datastore_pytables import Store

In [3]:
store = Store()
store.keys()

['/cont/min/CL_20191120_NYMEX_USD',
 '/cont/min/ES_20191220_GLOBEX_USD',
 '/cont/min/GC_20191227_NYMEX_USD',
 '/cont/min/GE_20191216_GLOBEX_USD',
 '/cont/min/NKD_20191212_GLOBEX_USD',
 '/cont/min/NQ_20191220_GLOBEX_USD',
 '/cont/min/YM_20191220_ECBOT_USD',
 '/cont/min/ZB_20191219_ECBOT_USD',
 '/cont/min/ZF_20191231_ECBOT_USD',
 '/cont/min/ZN_20191219_ECBOT_USD']

In [4]:
contract = store.read('/cont/min/NQ_20191220_GLOBEX_USD').sort_index()

In [5]:
table = bootstrap(contract,
          start = '20180201', 
          end = '20181231',
          paths=100)

In [6]:
func = partial(breakout_strategy, 
                  time_int = 30,
                  periods = [5, 10, 20, 40, 80, 160,],
                  ema_fast = 5,
                  ema_slow = 120,
                  atr_periods = 80,
                  sl_atr = 1,
                  )

In [7]:
results = m_proc(table, func)

In [8]:
res = pd.DataFrame([perf(i, output=False).stats for i in results])

In [9]:
res[res['Annual return'] > 0]['Annual return'].count()

97

In [10]:
res.sort_values('Annual return')

,Annual return,Cumulative returns,Annual volatility,Sharpe ratio,Calmar ratio,Stability,Max drawdown,Omega ratio,Sortino ratio,Skew,...,Avg gain/loss ratio,Position EV,Positions per day,Days per position,Actual avg. duration,Days,Positions,Trades,Monthly EV,Annual EV
94,-0.040554,-0.025462,0.170675,-0.158092,-0.338367,0.293131,-0.119851,0.967701,-0.243523,0.656012,...,1.745047,-0.452711,2.535032,0.394472,03:55:00,157,398,796,-24.100389,-289.204668
47,-0.021128,-0.013132,0.128202,-0.102957,-0.169332,0.362588,-0.124773,0.982334,-0.149102,0.279406,...,1.503035,-0.237906,2.416667,0.413793,04:20:00,156,377,754,-12.073746,-144.884950
1,-0.004132,-0.002609,0.126373,0.030090,-0.031417,0.062424,-0.131527,1.005822,0.043195,-0.161091,...,1.549594,-0.047585,2.459119,0.406650,04:36:00,159,391,782,-2.457341,-29.488097
69,0.020909,0.012975,0.127123,0.225816,0.286590,0.256651,-0.072956,1.040294,0.344049,0.383446,...,1.594493,0.229692,2.503185,0.399491,04:28:00,157,393,786,12.074200,144.890404
9,0.025080,0.015652,0.134020,0.251436,0.240007,0.391724,-0.104497,1.046875,0.372176,-0.025527,...,1.434635,0.267798,2.481013,0.403061,04:32:00,158,392,784,13.952624,167.431488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0.854610,0.465754,0.108577,5.749317,45.364290,0.945779,-0.018839,3.115760,15.179950,1.183507,...,1.987881,8.992806,2.185897,0.457478,04:48:00,156,341,682,412.804383,4953.652600
72,0.870599,0.473564,0.149485,4.267878,37.873393,0.976236,-0.022987,2.482968,12.133772,1.896055,...,2.384266,9.924072,2.211538,0.452174,05:05:00,156,345,690,460.896810,5530.761718
48,0.882218,0.475516,0.154422,4.176575,21.455688,0.931132,-0.041118,2.350681,9.877614,1.436880,...,2.173217,9.735391,2.277419,0.439093,05:04:00,155,353,706,465.602924,5587.235085
49,0.918462,0.504552,0.125253,5.269983,44.511025,0.985729,-0.020634,2.931671,14.294649,1.451019,...,2.166863,9.635466,2.272152,0.440111,04:37:00,158,359,717,459.758109,5517.097305


In [11]:
res['Sharpe ratio'].median()

2.696676253395944

In [12]:
res['Sharpe ratio'].mean()

2.6887858538519827

In [13]:
res[res['Annual return'] >=.2]['Annual return'].count()

86

In [14]:
res['Position EV'].median()

4.617069261358218

In [15]:
(res[res['Position EV'] < 2])['Annual return'].count()

13

In [16]:
(res[res['Position EV'] < .2])['Annual return'].count()

3

In [17]:
res['Monthly EV'].median()

236.40886838255508